# Imports

In [1]:
import pandas as pd
import h5py
import numpy as np
import tensorflow as tf
import h5py
import os

In [8]:
#Lanzar desde la carpeta LightX3ECGPrivate
import sys
sys.path.append('./automatic-ecg-diagnosis/')
# sys.path.append('./source/')
import predict

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

# Commands

In [2]:
!git clone https://github.com/antonior92/automatic-ecg-diagnosis.git

Cloning into 'automatic-ecg-diagnosis'...


In [7]:
#Comando windows
!py -m pip install tensorflow

^C


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!cd

e:\automatic-ecg-diagnosis-private


In [4]:
%cd automatic-ecg-diagnosis

e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis


In [2]:
!ls

"ls" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [11]:
!wget https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0 -O data.zip

--2023-05-02 18:21:58--  https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6024:18::a27d:4412
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: /s/raw/p3vd3plcbu9sf1o/data.zip [siguiendo]
--2023-05-02 18:21:58--  https://www.dropbox.com/s/raw/p3vd3plcbu9sf1o/data.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USPaOcJJ6qJM3mylVU6u86B3BR2k6jWUil7g29fotfKh0d6Kir86bKWE2Qd4ddmw2mvBrzpm5X-3y240SOomVSv8E0sCPCoBATC3cVbe2ZJF0MvPxg/file# [siguiendo]
--2023-05-02 18:21:59--  https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USP

In [12]:
!unzip data.zip

Archive:  data.zip
  inflating: data/ecg_tracings.hdf5  
replace data/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
!pip3 show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/tfg_luis_2223/.local/lib/python3.7/site-packages
Requires: google-pasta, astor, gast, keras-preprocessing, six, protobuf, wheel, keras-applications, absl-py, tensorboard, wrapt, grpcio, numpy, termcolor, tensorflow-estimator
Required-by: 


In [5]:
!python install -U pip

python: can't open file 'install': [Errno 2] No such file or directory


In [7]:
!python -v

# installing zipimport hook
import zipimport # builtin
# installed zipimport hook
# /usr/lib/python2.7/site.pyc matches /usr/lib/python2.7/site.py
import site # precompiled from /usr/lib/python2.7/site.pyc
# /usr/lib/python2.7/os.pyc matches /usr/lib/python2.7/os.py
import os # precompiled from /usr/lib/python2.7/os.pyc
import errno # builtin
import posix # builtin
# /usr/lib/python2.7/posixpath.pyc matches /usr/lib/python2.7/posixpath.py
import posixpath # precompiled from /usr/lib/python2.7/posixpath.pyc
# /usr/lib/python2.7/stat.pyc matches /usr/lib/python2.7/stat.py
import stat # precompiled from /usr/lib/python2.7/stat.pyc
# /usr/lib/python2.7/genericpath.pyc matches /usr/lib/python2.7/genericpath.py
import genericpath # precompiled from /usr/lib/python2.7/genericpath.pyc
# /usr/lib/python2.7/warnings.pyc matches /usr/lib/python2.7/warnings.py
import warnings # precompiled from /usr/lib/python2.7/warnings.pyc
# /usr/lib/python2.7/linecache.pyc matches /usr/lib/python2.7/linecache.

# Code

## crear hdf5

In [12]:
filename = 'data/val.hdf5'
f = h5py.File(filename, 'w')

In [13]:
folder_path = 'CasosNumpy'
df = pd.read_csv("data/annotations/val.csv")
group_name = 'tracings'
shape = (len(df['id'].values),4096,12)
dset = f.create_dataset(group_name,shape=shape)

count = 0
for file_name in os.listdir(folder_path):
    if(file_name[:5] in df['id'].values):
        if file_name.endswith('.npy'):
            # Cargar el archivo numpy
            arr = np.load(os.path.join(folder_path, file_name))[:,:4096]
            if arr.shape != (12, 4096):
                new_arr = np.zeros((12, 4096))
                new_arr[:arr.shape[0], :arr.shape[1]] = arr
                arr = new_arr
            #arr = tf.convert_to_tensor(arr, dtype=tf.float32)
            dset[count, :, :] = arr.transpose()
            count+=1
            
# Guardar el arreglo numpy en el grupo

In [14]:
f.close()

## Modificar val.csv a como lo tiene antonio

In [9]:
df = pd.read_csv("data/annotations/train.csv")
df = df.drop(['id','age','sex','r_count','length'],axis=1)
df.to_csv("data/annotations/train_CPSC.csv")

## Otros

In [2]:
args= "datasets/pred.hdf5"
with h5py.File(args, "r") as f:
    x = np.array(f['tracings'])

np.size(x[1])
x.shape

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'datasets/pred.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

# Train

In [33]:
!py train.py ./data/datos.hdf5 ./data/annotations/train_CPSC.csv

^C


In [31]:
!py train.py -h

usage: train.py [-h] [--val_split VAL_SPLIT] [--dataset_name DATASET_NAME]
                path_to_hdf5 path_to_csv

Train neural network.

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_csv           path to csv file containing annotations

optional arguments:
  -h, --help            show this help message and exit
  --val_split VAL_SPLIT
                        number between 0 and 1 determining how much of the
                        data is to be used for validation. The remaining is
                        used for validation. Default: 0.02
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings


# Predict

In [22]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/ecg.hdf5 model.hdf5 --output_file $PFOLDER/ecg_tracings.npy

Traceback (most recent call last):
  File "predict.py", line 28, in <module>
    seq = ECGSequence(args.path_to_hdf5, args.dataset_name, batch_size=args.bs)
  File "e:\automatic-ecg-diagnosis-private\automatic-ecg-diagnosis\datasets.py", line 24, in __init__
    self.f = h5py.File(path_to_hdf5, "r")
  File "C:\Users\quin-\AppData\Roaming\Python\Python37\site-packages\h5py\_hl\files.py", line 567, in __init__
    fid = make_fid(name, mode, userblock_size, fapl, fcpl, swmr=swmr)
  File "C:\Users\quin-\AppData\Roaming\Python\Python37\site-packages\h5py\_hl\files.py", line 231, in make_fid
    fid = h5f.open(name, flags, fapl=fapl)
  File "h5py\_objects.pyx", line 54, in h5py._objects.with_phil.wrapper
  File "h5py\_objects.pyx", line 55, in h5py._objects.with_phil.wrapper
  File "h5py\h5f.pyx", line 106, in h5py.h5f.open
FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = './data/ecg.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flag

In [ ]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/ecg_tracings2.hdf5 model.hdf5 --output_file $PFOLDER/ecg_tracings.npy

In [26]:
PFOLDER="./dnn_predicts"
DFOLDER="./data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/pred.hdf5 final_model.hdf5 --output_file $PFOLDER/pred.npy


1/1 [==============================] - 1s 611ms/step
Output predictions saved


2023-05-11 17:05:27.299509: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
!python predict.py -h

usage: predict.py [-h] [--dataset_name DATASET_NAME]
                  [--output_file OUTPUT_FILE] [-bs BS]
                  path_to_hdf5 path_to_model

Get performance on test set from hdf5

positional arguments:
  path_to_hdf5          path to hdf5 file containing tracings
  path_to_model         file containing training model.

optional arguments:
  -h, --help            show this help message and exit
  --dataset_name DATASET_NAME
                        name of the hdf5 dataset containing tracings
  --output_file OUTPUT_FILE
                        output csv file.
  -bs BS                Batch size.


In [17]:
pred = np.load('dnn_predicts/ecg_tracings.npy')
csv = pd.read_csv('data/annotations/dnn.csv')
pos = 22
print(pred[pos,1:])
print(csv.values[pos])


[2.5101798e-03 1.6017100e-05 6.6078037e-01 5.1851600e-04 2.2911260e-07]
[22  0  0  0  1  0  0]


In [23]:
pred = np.load('dnn_predicts/model_pred.npy')
pred[:,0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

In [42]:
pred = np.load('dnn_predicts/train.npy')
csv = pd.read_csv('data/annotations/train.csv')
pos = 0
print(pred[pos,1:])
print(csv.values[pos])


[9.6429495e-09 9.9344963e-01 5.0510238e-03 7.9479541e-06 4.1719411e-02
 2.9839315e-13 1.1956898e-03 2.2249950e-02 4.2172393e-18]
['A3483' 75 0 0 1 0 0 1 0 0 0 0 19 5000]


In [27]:
pred = np.load('dnn_predicts/pred.npy')
pos = 25
print(pred[pos,1:])

[9.7825001e-09 9.9343199e-01 5.0518359e-03 8.0328427e-06 4.1804649e-02
 3.0411887e-13 1.2021037e-03 2.2318605e-02 4.3300473e-18]
